# Distributed Training with GPUs on Cloud AI Platform

In this notebook we will walk through using Cloud AI Platform to perform distributed training using the `MirroredStrategy` found within `tf.keras`. This strategy will allow us to use the synchronous AllReduce strategy on a VM with multiple GPUs attached.

First we need to authenicate our Google Cloud account to be able to submit training jobs to AI Platform. We will do this via `gcloud auth login` to authenicate with OAuth2.

In [ ]:
# authorize your gcp account
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=hxv6UaFkCKRtQge51Cxt8IL9Gx8yOK&prompt=consent&access_type=offline&code_challenge=U9JdSfqDOPMFWiWdEX9YKsukCEmmh8HxYMFNawQyNGI&code_challenge_method=S256

Enter verification code: 4/1AY0e-g5z7sAlNXwXHiVZd5od8pfvvb1lEuIu4Q7_XF4O9k27zzAviW9yyhk

You are now logged in as [michaelabel.gcp@gmail.com].
Your current project is [michaelabel-gcp-training].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


Next we will configure our environment. Be sure to change the `PROJECT_ID` variable in the below cell to your Project ID. This will be the project to which the Cloud AI Platform resources will be billed. We will also create a bucket for our training artifacts (if it does not already exist).

In [ ]:
# config
import os
PROJECT_ID = 'michaelabel-gcp-training'  # TODO: Insert your Project ID here!
BUCKET = PROJECT_ID + '-accelerator-demo'
REGION = 'us-central1'
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["BUCKET"] = BUCKET

!gcloud config set project {PROJECT_ID}
!gsutil mb gs://{BUCKET}
!gcloud config set compute/region {REGION}

Updated property [core/project].
Creating gs://michaelabel-gcp-training-accelerator-demo/...
ServiceException: 409 Bucket michaelabel-gcp-training-accelerator-demo already exists.
Updated property [compute/region].


Since we are going to submit our training job to Cloud AI Platform, we need to create our trainer package. We will create the `train` directory for our package and create a blank `__init__.py` file so Python knows that this folder contains a package.

In [ ]:
!mkdir train
!touch train/__init__.py

Next we will create a module containing a function which will create our model. Note that we will be using the Fashion MNIST dataset. Since it's a small dataset, we will simply load it into memory for getting the parameters for our model.

Our model will be a DNN with only dense layers, applying dropout to each hidden layer. We will also use ReLU activation for all hidden layers.

In [ ]:
%%writefile train/model_definition.py
import tensorflow as tf
import numpy as np

# Get data

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# add empty color dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

def create_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=x_train.shape[1:]))
    model.add(tf.keras.layers.Dense(1028))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10))
    model.add(tf.keras.layers.Activation('softmax'))
    return model

Writing train/model_definition.py


Before we submit our training jobs to Cloud AI Platform, let's be sure our model runs locally. We will call the `model_definition` function to create our model and use `tf.keras.datasets.fashion_mnist.load_data()` to import the Fashion MNIST dataset.

In [ ]:
import os
import os
import time
import tensorflow as tf
import numpy as np
from train import model_definition

#Get data

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# add empty color dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

def create_dataset(X, Y, epochs, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.repeat(epochs).batch(batch_size, drop_remainder=True)
    return dataset

ds_train = create_dataset(x_train, y_train, 1, 5000)
ds_test = create_dataset(x_test, y_test, 1, 1000)

model = model_definition.create_model()

model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
  loss='sparse_categorical_crossentropy',
  metrics=['sparse_categorical_accuracy'])
    
start = time.time()

model.fit(
    ds_train,
    validation_data=ds_test, 
    verbose=1
)
print("Training time without GPUs locally: {}".format(time.time() - start))

12/12 [==============================] - 9s 729ms/step - loss: 75.6025 - sparse_categorical_accuracy: 0.3110 - val_loss: 1.8982 - val_sparse_categorical_accuracy: 0.6896
Training time without GPUs locally: 10.302881956100464




# Train on multiple GPUs/CPUs with MultiWorkerMirrored Strategy


That took a few minutes to train our model for 20 epochs. Let's see how we can do better using Cloud AI Platform. We will be leveraging the `MultiWorkerMirroredStrategy` supplied in `tf.distribute`. The main difference between this code, and the code from the local test is that we need to compile the model within the scope of the strategy. When we do this our training op will use information stored in the `TF_CONFIG` variable to assign ops to the various devices for the AllReduce strategy. 

After the training process finishes, we will print out the time spent training. Since it takes a few minutes to spin up the resources being used for training on Cloud AI Platform, and this time can vary, we want a consistent measure of how long training took.

Note: When we train models on Cloud AI Platform, the `TF_CONFIG` variable is automatically set. So we do not need to worry about adjusting based what cluster configuration we use.

In [ ]:
%%writefile train/train_mult_worker_mirrored.py
import os
import os
import time
import tensorflow as tf
import numpy as np
from . import model_definition

strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

#Get data

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# add empty color dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

def create_dataset(X, Y, epochs, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.repeat(epochs).batch(batch_size, drop_remainder=True)
    return dataset

ds_train = create_dataset(x_train, y_train, 20, 5000)
ds_test = create_dataset(x_test, y_test, 1, 1000)

print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    model = model_definition.create_model()
    model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])
    
start = time.time()

model.fit(
    ds_train,
    validation_data=ds_test, 
    verbose=2
)
print("Training time with multiple GPUs: {}".format(time.time() - start))

Writing train/train_mult_worker_mirrored.py


First we will train a model without using GPUs to give us a baseline. We will use a consistent format throughout the trials. We will define a `config.yaml` file to contain our cluster configuration and the pass this file in as the value of a command-line argument `--config`.

In our first example, we will use a single `n1-highcpu-16` VM.

In [ ]:
%%writefile config.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: n1-highcpu-16

Writing config.yaml


In [ ]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="cpu_only_fashion_minst_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://$BUCKET \
  --package-path=train \
  --module-name=train.train_mult_worker_mirrored \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --region=us-west1 \
  --config config.yaml

jobId: cpu_only_fashion_minst_20201207_140943
state: QUEUED


Job [cpu_only_fashion_minst_20201207_140943] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe cpu_only_fashion_minst_20201207_140943

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs cpu_only_fashion_minst_20201207_140943


If we go through the logs, we see that the training took around 30 seconds (though the exact number may vary for your case). Let's now attach two Nvidia Tesla K80 GPUs and rerun the training job.

In [ ]:
%%writefile config.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: n1-highcpu-16
  masterConfig:
    acceleratorConfig:
      count: 2
      type: NVIDIA_TESLA_K80

Overwriting config.yaml


In [ ]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="multi_gpu_fashion_minst_2gpu_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://$BUCKET \
  --package-path=train \
  --module-name=train.train_mult_worker_mirrored \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --region=us-west1 \
  --config config.yaml

jobId: multi_gpu_fashion_minst_2gpu_20201207_140946
state: QUEUED


Job [multi_gpu_fashion_minst_2gpu_20201207_140946] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe multi_gpu_fashion_minst_2gpu_20201207_140946

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs multi_gpu_fashion_minst_2gpu_20201207_140946


That was a lot faster! On my end it took around 6 seconds to train the model. Let's keep going and add more GPUs!

In [ ]:
%%writefile config.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: n1-highcpu-16
  masterConfig:
    acceleratorConfig:
      count: 4
      type: NVIDIA_TESLA_K80

Overwriting config.yaml


In [ ]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="multi_gpu_fashion_minst_4gpu_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://$BUCKET \
  --package-path=train \
  --module-name=train.train_mult_worker_mirrored \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --region=us-west1 \
  --config config.yaml

jobId: multi_gpu_fashion_minst_4gpu_20201207_140948
state: QUEUED


Job [multi_gpu_fashion_minst_4gpu_20201207_140948] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe multi_gpu_fashion_minst_4gpu_20201207_140948

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs multi_gpu_fashion_minst_4gpu_20201207_140948


Wait...what happened? It took around 10 seconds to train the model in this case. It was faster than no GPUs, but why was it slower than 2 GPUs? If you rerun this job with 8 GPUs you'll actually see it takes just as long as using no GPUs!

The answer is in our input pipeline. In short, the I/O involved in using more GPUs started to outweigh the benefits of having more available devices. We can try to improve our input pipelines to overcome this (e.g. using caching, adjusting batch size, etc.). 

Finally, what would the `config.yaml` look like if we wanted to use multiple VMs as well? We include that example here, but will not run the training job. Note that we don't have to change any code in our trainer package! However, we may want to consider our input pipeline to really take advantage of the larger cluster.

In [ ]:
%%writefile config.yaml
trainingInput:
  scaleTier: CUSTOM
  # Configure a master worker with 4 T4 GPUs
  masterType: n1-highcpu-16
  masterConfig:
    acceleratorConfig:
      count: 4
      type: NVIDIA_TESLA_T4
  # Configure 9 workers, each with 4 T4 GPUs
  workerCount: 9
  workerType: n1-highcpu-16
  workerConfig:
    acceleratorConfig:
      count: 4
      type: NVIDIA_TESLA_T4